<a href="https://colab.research.google.com/github/zacknn/MachineLearning/blob/main/models/neural_networks/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:08<00:00, 19.7MB/s]


In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# --- STEP 2: Define the Network ---
class CifarCNN(nn.Module):
    def __init__(self):
        super(CifarCNN, self).__init__()

        # 1. Convolution: 3 Input Channels (RGB) -> 8 Output Features
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3)

        # 2. Pool: Shrink by 2
        self.pool = nn.MaxPool2d(2, 2)

        # 3. Linear: 8 channels * 15 * 15 image size -> 10 Output Classes
        self.fc1 = nn.Linear(8 * 15 * 15, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 8 * 15 * 15) # Flatten
        x = self.fc1(x)
        return x

net = CifarCNN()

# --- STEP 3: The "Teacher" (Loss & Optimizer) ---
# Loss: Calculates how wrong the guess was
criterion = nn.CrossEntropyLoss()

# Optimizer: Adjusts the weights to make the guess better next time
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [10]:
# --- STEP 4: The Training Loop ---
print("Start Training...")

# Loop over the dataset for 10 epochs
for epoch in range(10): # Increased from 2 to 10 epochs
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        # A. Zero the gradients (reset for this batch)
        optimizer.zero_grad()

        # B. Forward Pass (Make a guess)
        outputs = net(inputs)

        # C. Calculate Loss (How wrong was the guess?)
        loss = criterion(outputs, labels)

        # D. Backward Pass (Calculate corrections)
        loss.backward()

        # E. Optimize (Apply corrections)
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[Epoch: {epoch + 1}, Batch: {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Start Training...
[Epoch: 1, Batch: 2000] loss: 1.251
[Epoch: 1, Batch: 4000] loss: 1.263
[Epoch: 1, Batch: 6000] loss: 1.259
[Epoch: 1, Batch: 8000] loss: 1.268
[Epoch: 1, Batch: 10000] loss: 1.254
[Epoch: 1, Batch: 12000] loss: 1.272
[Epoch: 2, Batch: 2000] loss: 1.196
[Epoch: 2, Batch: 4000] loss: 1.226
[Epoch: 2, Batch: 6000] loss: 1.229
[Epoch: 2, Batch: 8000] loss: 1.218
[Epoch: 2, Batch: 10000] loss: 1.228
[Epoch: 2, Batch: 12000] loss: 1.242
[Epoch: 3, Batch: 2000] loss: 1.178
[Epoch: 3, Batch: 4000] loss: 1.164
[Epoch: 3, Batch: 6000] loss: 1.208
[Epoch: 3, Batch: 8000] loss: 1.224
[Epoch: 3, Batch: 10000] loss: 1.220
[Epoch: 3, Batch: 12000] loss: 1.211
[Epoch: 4, Batch: 2000] loss: 1.156
[Epoch: 4, Batch: 4000] loss: 1.157
[Epoch: 4, Batch: 6000] loss: 1.182
[Epoch: 4, Batch: 8000] loss: 1.191
[Epoch: 4, Batch: 10000] loss: 1.172
[Epoch: 4, Batch: 12000] loss: 1.223
[Epoch: 5, Batch: 2000] loss: 1.123
[Epoch: 5, Batch: 4000] loss: 1.169
[Epoch: 5, Batch: 6000] loss: 1.150
[E

In [13]:
# Get a few random test images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Ask the network to guess
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Ground Truth: ', ' '.join(f'{classes[labels[j]]}' for j in range(4)))
print('Prediction:   ', ' '.join(f'{classes[predicted[j]]}' for j in range(4)))

Ground Truth:  horse deer dog horse
Prediction:    horse deer dog horse
